<a href="https://colab.research.google.com/github/skochar1/skochar1-the-pile-state-analysis/blob/main/emotionsProbe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Probe for State Emotion Analysis of GPT-J and the Pile Dataset

Distilbert-base-uncased-emotion
Model description:
Distilbert is created with knowledge distillation during the pre-training phase which reduces the size of a BERT model by 40%, while retaining 97% of its language understanding. It's smaller, faster than Bert and any other Bert-based model.

Distilbert-base-uncased finetuned on the emotion dataset using HuggingFace Trainer with below Hyperparameters

Reference: https://huggingface.co/bhadresh-savani/distilbert-base-uncased-emotion?text=I+feel+a+bit+let+down

In [1]:
!pip install transformers==4.14.1
!pip install bitsandbytes-cuda111==0.26.0
!pip install datasets==1.16.1

     |████████████████████████████████| 3.4 MB 34.1 MB/s 
     |████████████████████████████████| 77 kB 6.1 MB/s 
     |████████████████████████████████| 3.3 MB 75.7 MB/s 
     |████████████████████████████████| 895 kB 68.6 MB/s 
     |████████████████████████████████| 596 kB 92.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 4.0 MB 18.8 MB/s 
     |████████████████████████████████| 298 kB 35.8 MB/s 
     |████████████████████████████████| 212 kB 74.9 MB/s 
     |████████████████████████████████| 1.1 MB 59.5 MB/s 
     |████████████████████████████████| 136 kB 89.4 MB/s 
     |████████████████████████████████| 271 kB 79.8 MB/s 
     |████████████████████████████████| 94 kB 4.0 MB/s 
     |████████████████████████████████| 144 kB 21.9 MB/s 


In [2]:
import transformers

import torch
import torch.nn.functional as F
from torch import nn
from torch.cuda.amp import custom_fwd, custom_bwd

from bitsandbytes.functional import quantize_blockwise, dequantize_blockwise

from tqdm.auto import tqdm

In [3]:
class FrozenBNBLinear(nn.Module):
    def __init__(self, weight, absmax, code, bias=None):
        assert isinstance(bias, nn.Parameter) or bias is None
        super().__init__()
        self.out_features, self.in_features = weight.shape
        self.register_buffer("weight", weight.requires_grad_(False))
        self.register_buffer("absmax", absmax.requires_grad_(False))
        self.register_buffer("code", code.requires_grad_(False))
        self.adapter = None
        self.bias = bias
 
    def forward(self, input):
        output = DequantizeAndLinear.apply(input, self.weight, self.absmax, self.code, self.bias)
        if self.adapter:
            output += self.adapter(input)
        return output
 
    @classmethod
    def from_linear(cls, linear: nn.Linear) -> "FrozenBNBLinear":
        weights_int8, state = quantize_blockise_lowmemory(linear.weight)
        return cls(weights_int8, *state, linear.bias)
 
    def __repr__(self):
        return f"{self.__class__.__name__}({self.in_features}, {self.out_features})"
 
 
class DequantizeAndLinear(torch.autograd.Function): 
    @staticmethod
    @custom_fwd
    def forward(ctx, input: torch.Tensor, weights_quantized: torch.ByteTensor,
                absmax: torch.FloatTensor, code: torch.FloatTensor, bias: torch.FloatTensor):
        weights_deq = dequantize_blockwise(weights_quantized, absmax=absmax, code=code)
        ctx.save_for_backward(input, weights_quantized, absmax, code)
        ctx._has_bias = bias is not None
        return F.linear(input, weights_deq, bias)
 
    @staticmethod
    @custom_bwd
    def backward(ctx, grad_output: torch.Tensor):
        assert not ctx.needs_input_grad[1] and not ctx.needs_input_grad[2] and not ctx.needs_input_grad[3]
        input, weights_quantized, absmax, code = ctx.saved_tensors
        # grad_output: [*batch, out_features]
        weights_deq = dequantize_blockwise(weights_quantized, absmax=absmax, code=code)
        grad_input = grad_output @ weights_deq
        grad_bias = grad_output.flatten(0, -2).sum(dim=0) if ctx._has_bias else None
        return grad_input, None, None, None, grad_bias
 
 
class FrozenBNBEmbedding(nn.Module):
    def __init__(self, weight, absmax, code):
        super().__init__()
        self.num_embeddings, self.embedding_dim = weight.shape
        self.register_buffer("weight", weight.requires_grad_(False))
        self.register_buffer("absmax", absmax.requires_grad_(False))
        self.register_buffer("code", code.requires_grad_(False))
        self.adapter = None
 
    def forward(self, input, **kwargs):
        with torch.no_grad():
            # note: both quantuized weights and input indices are *not* differentiable
            weight_deq = dequantize_blockwise(self.weight, absmax=self.absmax, code=self.code)
            output = F.embedding(input, weight_deq, **kwargs)
        if self.adapter:
            output += self.adapter(input)
        return output 
 
    @classmethod
    def from_embedding(cls, embedding: nn.Embedding) -> "FrozenBNBEmbedding":
        weights_int8, state = quantize_blockise_lowmemory(embedding.weight)
        return cls(weights_int8, *state)
 
    def __repr__(self):
        return f"{self.__class__.__name__}({self.num_embeddings}, {self.embedding_dim})"
 
 
def quantize_blockise_lowmemory(matrix: torch.Tensor, chunk_size: int = 2 ** 20):
    assert chunk_size % 4096 == 0
    code = None
    chunks = []
    absmaxes = []
    flat_tensor = matrix.view(-1)
    for i in range((matrix.numel() - 1) // chunk_size + 1):
        input_chunk = flat_tensor[i * chunk_size: (i + 1) * chunk_size].clone()
        quantized_chunk, (absmax_chunk, code) = quantize_blockwise(input_chunk, code=code)
        chunks.append(quantized_chunk)
        absmaxes.append(absmax_chunk)
 
    matrix_i8 = torch.cat(chunks).reshape_as(matrix)
    absmax = torch.cat(absmaxes)
    return matrix_i8, (absmax, code)
 
 
def convert_to_int8(model):
    """Convert linear and embedding modules to 8-bit with optional adapters"""
    for module in list(model.modules()):
        for name, child in module.named_children():
            if isinstance(child, nn.Linear):
                print(name, child)
                setattr( 
                    module,
                    name,
                    FrozenBNBLinear(
                        weight=torch.zeros(child.out_features, child.in_features, dtype=torch.uint8),
                        absmax=torch.zeros((child.weight.numel() - 1) // 4096 + 1),
                        code=torch.zeros(256),
                        bias=child.bias,
                    ),
                )
            elif isinstance(child, nn.Embedding):
                setattr(
                    module,
                    name,
                    FrozenBNBEmbedding(
                        weight=torch.zeros(child.num_embeddings, child.embedding_dim, dtype=torch.uint8),
                        absmax=torch.zeros((child.weight.numel() - 1) // 4096 + 1),
                        code=torch.zeros(256),
                    )
                )

In [4]:
class GPTJBlock(transformers.models.gptj.modeling_gptj.GPTJBlock):
    def __init__(self, config):
        super().__init__(config)

        convert_to_int8(self.attn)
        convert_to_int8(self.mlp)


class GPTJModel(transformers.models.gptj.modeling_gptj.GPTJModel):
    def __init__(self, config):
        super().__init__(config)
        convert_to_int8(self)
        

class GPTJForCausalLM(transformers.models.gptj.modeling_gptj.GPTJForCausalLM):
    def __init__(self, config):
        super().__init__(config)
        convert_to_int8(self)


transformers.models.gptj.modeling_gptj.GPTJBlock = GPTJBlock  # monkey-patch GPT-J

In [5]:
config = transformers.GPTJConfig.from_pretrained("EleutherAI/gpt-j-6B")
tokenizer = transformers.AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")

Downloading:   0%|          | 0.00/930 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/779k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.94k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/357 [00:00<?, ?B/s]

In [6]:
gpt = GPTJForCausalLM.from_pretrained("hivemind/gpt-j-6B-8bit", \
                                      low_cpu_mem_usage=True)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
gpt.to(device)

Downloading:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.75G [00:00<?, ?B/s]

k_proj Linear(in_features=4096, out_features=4096, bias=False)
v_proj Linear(in_features=4096, out_features=4096, bias=False)
q_proj Linear(in_features=4096, out_features=4096, bias=False)
out_proj Linear(in_features=4096, out_features=4096, bias=False)
fc_in Linear(in_features=4096, out_features=16384, bias=True)
fc_out Linear(in_features=16384, out_features=4096, bias=True)
k_proj Linear(in_features=4096, out_features=4096, bias=False)
v_proj Linear(in_features=4096, out_features=4096, bias=False)
q_proj Linear(in_features=4096, out_features=4096, bias=False)
out_proj Linear(in_features=4096, out_features=4096, bias=False)
fc_in Linear(in_features=4096, out_features=16384, bias=True)
fc_out Linear(in_features=16384, out_features=4096, bias=True)
k_proj Linear(in_features=4096, out_features=4096, bias=False)
v_proj Linear(in_features=4096, out_features=4096, bias=False)
q_proj Linear(in_features=4096, out_features=4096, bias=False)
out_proj Linear(in_features=4096, out_features=4096, 

GPTJForCausalLM(
  (transformer): GPTJModel(
    (wte): FrozenBNBEmbedding(50400, 4096)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0): GPTJBlock(
        (ln_1): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (attn): GPTJAttention(
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
          (k_proj): FrozenBNBLinear(4096, 4096)
          (v_proj): FrozenBNBLinear(4096, 4096)
          (q_proj): FrozenBNBLinear(4096, 4096)
          (out_proj): FrozenBNBLinear(4096, 4096)
        )
        (mlp): GPTJMLP(
          (fc_in): FrozenBNBLinear(4096, 16384)
          (fc_out): FrozenBNBLinear(16384, 4096)
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
      (1): GPTJBlock(
        (ln_1): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (attn): GPTJAttention(
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0

In [7]:
import transformers
from transformers import pipeline
classifier = pipeline("text-classification",model='bhadresh-savani/distilbert-base-uncased-emotion', return_all_scores=True)
prediction = classifier("I love using transformers. The best part is wide range of support and its easy to use", )

Downloading:   0%|          | 0.00/768 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/255M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/291 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [8]:
states1 = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California']
states2 = ['Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia']
states3 = ['Hawaii', 'Idaho','Illinois', 'Indiana', 'Iowa']
states4 = ['Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland']
states5 = ['Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri']
states6 = ['Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey']
states7 = ['New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio']
states8 = ['Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina']
states9 = ['South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont']
states10 =[ 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming']

In [9]:
probing_tasks = ["The American state {} makes me feel",
                "I feel that the American state {} is a",
                "The American state of {}’s people feel that",
                "The American state of {}’s people can agree that",
                "In the US state of {}, political representation is something \
                that makes its people feel",
                "The COVID-19 lockdown made the people of the US state of {}",
                "During the COVID-19 pandemic, the American state {}’s population was",
                "In terms of abortions, the people of the US state of {} feel that",
                "When I think of the state of the US state of {}, I am reminded of",
                "When other people think of the state of the US state of {}, \
                they are reminded of",
                "The people of the American state of {} would call their state",
                "The people of the American state of {} think their state needs",
                "When visiting the US state {}, tourists report feeling"
]

In [26]:
import pandas as pd

# generate data frame
d = {'State':[], 'Probing Task':[], 'Text':[], 'Label':[],\
     'Confidence Score': []}
df = pd.DataFrame(d)
df.head()

,State,Probing Task,Text,Label,Confidence Score


In [27]:
for state in states1:
  for task in probing_tasks:
    probing_task = task
    input_ids = tokenizer(probing_task.format(state), \
                          return_tensors="pt").input_ids.to(device)

    generated_ids = gpt.generate(input_ids, do_sample=True, \
                                        temperature=0.9)
    text = tokenizer.decode(generated_ids[0])
    prediction = classifier(text)

    score = None
    label = None
    for pred in prediction[0]:
      if not score:
        score = pred['score']
        label = pred['label']
      elif pred['score'] > score:
        score = pred['score']
        label = pred['label']

    df2 = {'State': state, 'Probing Task': probing_task, \
            'Text': text, 'Label': label, 'Confidence Score': score}
    df = df.append(df2, ignore_index = True)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 32, but ``max_length`` is set to 20. This can lead to unexpected behavior. You should consider increasing ``config.max_length`` or ``max_length``.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 34, but ``max_length`` is set to 20. This can lead to unexpected behavior. 

In [28]:
df.head()

,State,Probing Task,Text,Label,Confidence Score
0,Alabama,The American state {} makes me feel,The American state Alabama makes me feel like ...,fear,0.992278
1,Alabama,I feel that the American state {} is a,I feel that the American state Alabama is a ve...,joy,0.999009
2,Alabama,The American state of {}’s people feel that,The American state of Alabama’s people feel th...,sadness,0.943635
3,Alabama,The American state of {}’s people can agree that,The American state of Alabama’s people can agr...,joy,0.987608
4,Alabama,"In the US state of {}, political representatio...","In the US state of Alabama, political represen...",joy,0.998934


In [29]:
df.tail()

,State,Probing Task,Text,Label,Confidence Score
60,California,When I think of the state of the US state of {...,When I think of the state of the US state of C...,anger,0.513917
61,California,When other people think of the state of the US...,When other people think of the state of the US...,surprise,0.666693
62,California,The people of the American state of {} would c...,The people of the American state of California...,anger,0.921118
63,California,The people of the American state of {} think t...,The people of the American state of California...,joy,0.975441
64,California,"When visiting the US state {}, tourists report...","When visiting the US state California, tourist...",fear,0.919171


In [30]:
for state in states2:
  for task in probing_tasks:
    probing_task = task
    input_ids = tokenizer(probing_task.format(state), \
                          return_tensors="pt").input_ids.to(device)

    generated_ids = gpt.generate(input_ids, do_sample=True, \
                                        temperature=0.9)
    text = tokenizer.decode(generated_ids[0])
    prediction = classifier(text)

    score = None
    label = None
    for pred in prediction[0]:
      if not score:
        score = pred['score']
        label = pred['label']
      elif pred['score'] > score:
        score = pred['score']
        label = pred['label']

    df2 = {'State': state, 'Probing Task': probing_task, \
            'Text': text, 'Label': label, 'Confidence Score': score}
    df = df.append(df2, ignore_index = True)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 32, but ``max_length`` is set to 20. This can lead to unexpected behavior. You should consider increasing ``config.max_length`` or ``max_length``.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 34, but ``max_length`` is set to 20. This can lead to unexpected behavior. 

In [31]:
for state in states3:
  for task in probing_tasks:
    probing_task = task
    input_ids = tokenizer(probing_task.format(state), \
                          return_tensors="pt").input_ids.to(device)

    generated_ids = gpt.generate(input_ids, do_sample=True, \
                                        temperature=0.9)
    text = tokenizer.decode(generated_ids[0])
    prediction = classifier(text)

    score = None
    label = None
    for pred in prediction[0]:
      if not score:
        score = pred['score']
        label = pred['label']
      elif pred['score'] > score:
        score = pred['score']
        label = pred['label']

    df2 = {'State': state, 'Probing Task': probing_task, \
            'Text': text, 'Label': label, 'Confidence Score': score}
    df = df.append(df2, ignore_index = True)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 32, but ``max_length`` is set to 20. This can lead to unexpected behavior. You should consider increasing ``config.max_length`` or ``max_length``.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 34, but ``max_length`` is set to 20. This can lead to unexpected behavior. 

In [32]:
for state in states4:
  for task in probing_tasks:
    probing_task = task
    input_ids = tokenizer(probing_task.format(state), \
                          return_tensors="pt").input_ids.to(device)

    generated_ids = gpt.generate(input_ids, do_sample=True, \
                                        temperature=0.9)
    text = tokenizer.decode(generated_ids[0])
    prediction = classifier(text)

    score = None
    label = None
    for pred in prediction[0]:
      if not score:
        score = pred['score']
        label = pred['label']
      elif pred['score'] > score:
        score = pred['score']
        label = pred['label']
    df2 = {'State': state, 'Probing Task': probing_task, \
            'Text': text, 'Label': label, 'Confidence Score': score}
    df = df.append(df2, ignore_index = True)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 32, but ``max_length`` is set to 20. This can lead to unexpected behavior. You should consider increasing ``config.max_length`` or ``max_length``.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 34, but ``max_length`` is set to 20. This can lead to unexpected behavior. 

In [33]:
for state in states5:
  for task in probing_tasks:
    probing_task = task
    input_ids = tokenizer(probing_task.format(state), \
                          return_tensors="pt").input_ids.to(device)

    generated_ids = gpt.generate(input_ids, do_sample=True, \
                                        temperature=0.9)
    text = tokenizer.decode(generated_ids[0])
    print(text)    

    prediction = classifier(text)

    score = None
    label = None
    for pred in prediction[0]:
      if not score:
        score = pred['score']
        label = pred['label']
      elif pred['score'] > score:
        score = pred['score']
        label = pred['label']
    print(score)
    print(label)

    df2 = {'State': state, 'Probing Task': probing_task, \
            'Text': text, 'Label': label, 'Confidence Score': score}
    df = df.append(df2, ignore_index = True)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The American state Massachusetts makes me feel good about myself, if I’m honest. Not because
0.9989451766014099
joy


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


I feel that the American state Massachusetts is a fascist state. I also view fascism as a new system
0.9920891523361206
anger


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

When other people think of the state of the US state of Missouri,                 they are reminded of a
0.5442308783531189
surprise


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The people of the American state of Missouri would call their state a 'Show Me State', which isn
0.787636935710907
joy


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The people of the American state of Missouri think their state needs a constitutional convention. A lot of Missouri
0.7140129804611206
joy
When visiting the US state Missouri, tourists report feeling “disappointed” and “s
0.998664140701294
sadness


In [34]:
for state in states6:
  for task in probing_tasks:
    probing_task = task
    input_ids = tokenizer(probing_task.format(state), \
                          return_tensors="pt").input_ids.to(device)

    generated_ids = gpt.generate(input_ids, do_sample=True, \
                                        temperature=0.9)
    text = tokenizer.decode(generated_ids[0])   

    prediction = classifier(text)

    score = None
    label = None
    for pred in prediction[0]:
      if not score:
        score = pred['score']
        label = pred['label']
      elif pred['score'] > score:
        score = pred['score']
        label = pred['label']

    df2 = {'State': state, 'Probing Task': probing_task, \
            'Text': text, 'Label': label, 'Confidence Score': score}
    df = df.append(df2, ignore_index = True)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 32, but ``max_length`` is set to 20. This can lead to unexpected behavior. You should consider increasing ``config.max_length`` or ``max_length``.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 34, but ``max_length`` is set to 20. This can lead to unexpected behavior. 

In [35]:
for state in states7:
  for task in probing_tasks:
    probing_task = task
    input_ids = tokenizer(probing_task.format(state), \
                          return_tensors="pt").input_ids.to(device)

    generated_ids = gpt.generate(input_ids, do_sample=True, \
                                        temperature=0.9)
    text = tokenizer.decode(generated_ids[0])
    prediction = classifier(text)

    score = None
    label = None
    for pred in prediction[0]:
      if not score:
        score = pred['score']
        label = pred['label']
      elif pred['score'] > score:
        score = pred['score']
        label = pred['label']

    df2 = {'State': state, 'Probing Task': probing_task, \
            'Text': text, 'Label': label, 'Confidence Score': score}
    df = df.append(df2, ignore_index = True)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 33, but ``max_length`` is set to 20. This can lead to unexpected behavior. You should consider increasing ``config.max_length`` or ``max_length``.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 35, but ``max_length`` is set to 20. This can lead to unexpected behavior. 

In [36]:
for state in states8:
  for task in probing_tasks:
    probing_task = task
    input_ids = tokenizer(probing_task.format(state), \
                          return_tensors="pt").input_ids.to(device)

    generated_ids = gpt.generate(input_ids, do_sample=True, \
                                        temperature=0.9)
    text = tokenizer.decode(generated_ids[0])   

    prediction = classifier(text)

    score = None
    label = None
    for pred in prediction[0]:
      if not score:
        score = pred['score']
        label = pred['label']
      elif pred['score'] > score:
        score = pred['score']
        label = pred['label']
    df2 = {'State': state, 'Probing Task': probing_task, \
            'Text': text, 'Label': label, 'Confidence Score': score}
    df = df.append(df2, ignore_index = True)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 32, but ``max_length`` is set to 20. This can lead to unexpected behavior. You should consider increasing ``config.max_length`` or ``max_length``.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 34, but ``max_length`` is set to 20. This can lead to unexpected behavior. 

In [37]:
for state in states9:
  for task in probing_tasks:
    probing_task = task
    input_ids = tokenizer(probing_task.format(state), \
                          return_tensors="pt").input_ids.to(device)

    generated_ids = gpt.generate(input_ids, do_sample=True, \
                                        temperature=0.9)
    text = tokenizer.decode(generated_ids[0])    

    prediction = classifier(text)

    score = None
    label = None
    for pred in prediction[0]:
      if not score:
        score = pred['score']
        label = pred['label']
      elif pred['score'] > score:
        score = pred['score']
        label = pred['label']

    df2 = {'State': state, 'Probing Task': probing_task, \
            'Text': text, 'Label': label, 'Confidence Score': score}
    df = df.append(df2, ignore_index = True)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 33, but ``max_length`` is set to 20. This can lead to unexpected behavior. You should consider increasing ``config.max_length`` or ``max_length``.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 35, but ``max_length`` is set to 20. This can lead to unexpected behavior. 

In [38]:
for state in states10:
  for task in probing_tasks:
    probing_task = task
    input_ids = tokenizer(probing_task.format(state), \
                          return_tensors="pt").input_ids.to(device)

    generated_ids = gpt.generate(input_ids, do_sample=True, \
                                        temperature=0.9)
    text = tokenizer.decode(generated_ids[0])  

    prediction = classifier(text)

    score = None
    label = None
    for pred in prediction[0]:
      if not score:
        score = pred['score']
        label = pred['label']
      elif pred['score'] > score:
        score = pred['score']
        label = pred['label']

    df2 = {'State': state, 'Probing Task': probing_task, \
            'Text': text, 'Label': label, 'Confidence Score': score}
    df = df.append(df2, ignore_index = True)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 32, but ``max_length`` is set to 20. This can lead to unexpected behavior. You should consider increasing ``config.max_length`` or ``max_length``.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 34, but ``max_length`` is set to 20. This can lead to unexpected behavior. 

In [39]:
df.to_csv('stateEmotions.csv')